# Analyzing your model with TensorFlow Model Analysis and the What-If Tool

In [1]:
import tensorflow_model_analysis as tfma
import tensorflow as tf

import sys
import os

# stop tf warnings going everywhere
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [2]:
# add project to the python path
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [3]:
_EVAL_DATA_FILE = 'data_tfrecord-00000-of-00001'
_MODEL_DIR = 'serving_model_dir_2000_steps/'

## TFMA

In [4]:
eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=_MODEL_DIR, tags=[tf.saved_model.SERVING])

In [5]:
slices = [tfma.slicer.SingleSliceSpec(),
          tfma.slicer.SingleSliceSpec(columns=['product'])]

In [ ]:
eval_config=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
        metrics_specs=[
              tfma.MetricsSpec(metrics=[
                  tfma.MetricConfig(class_name='BinaryAccuracy'),
                  tfma.MetricConfig(class_name='ExampleCount'),
                  tfma.MetricConfig(class_name='FalsePositives'),
                  tfma.MetricConfig(class_name='TruePositives'),
                  tfma.MetricConfig(class_name='FalseNegatives'),
                  tfma.MetricConfig(class_name='TrueNegatives')
              ])])

In [ ]:
eval_config_viz=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
        metrics_specs=[
              tfma.MetricsSpec(metrics=[
                  tfma.MetricConfig(class_name='BinaryAccuracy'),
                  tfma.MetricConfig(class_name='AUC'),
                  tfma.MetricConfig(class_name='ExampleCount'),
                  tfma.MetricConfig(class_name='Precision'),
                  tfma.MetricConfig(class_name='Recall')
              ])])

In [ ]:
tfma.view.render_slicing_metrics(eval_result)

In [ ]:
tfma.view.render_slicing_metrics(eval_result, slicing_spec=slices[1])

In [ ]:
eval_config_plot=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
    metrics_specs=tfma.metrics.specs_from_metrics([
        tfma.metrics.ConfusionMatrixPlot(),
    ]))

In [ ]:
eval_result = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model,
    eval_config=eval_config_plot,
    data_location=_EVAL_DATA_FILE,
    output_path="./eval_result_viz",
    file_format='tfrecords',
    slice_spec = slices)

In [ ]:
# does not work right now, javascript errors
tfma.view.render_plot(eval_result)

## Adding a custom metric

In [ ]:
# try to do F1 score

In [ ]:
# custom Keras metric
# https://github.com/tensorflow/model-analysis/blob/master/g3doc/metrics.md
class MyMetric(tf.keras.metrics.Mean):

  def __init__(self, name='my_metric', dtype=None):
    super(MyMetric, self).__init__(name=name, dtype=dtype)

  def update_state(self, y_true, y_pred, sample_weight=None):
    return super(MyMetric, self).update_state(
        y_pred, sample_weight=sample_weight)

In [ ]:
# https://github.com/tensorflow/model-analysis/blob/master/g3doc/post_export_metrics.md

## Compare 2 models

In [ ]:
eval_shared_model_2 = tfma.default_eval_shared_model(
    eval_saved_model_path='serving_model_dir_150_steps/', tags=[tf.saved_model.SERVING])

eval_result_2 = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model_2,
    eval_config=eval_config,
    data_location=_EVAL_DATA_FILE,
    output_path="./eval_result_150_steps",
    file_format='tfrecords',
    slice_spec = slices)

In [ ]:
tfma.view.render_slicing_metrics(eval_result_2)

In [ ]:
eval_results_from_disk = tfma.load_eval_results(
    ['./eval_result_2000_steps','./eval_result_150_steps'], tfma.constants.MODEL_CENTRIC_MODE)

In [ ]:
#tfma.view.render_time_series(eval_results_from_disk, slices[0])
# many js errors

## Validating against thresholds

In [ ]:
eval_config_threshold=tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
    slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
    metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(class_name='BinaryAccuracy'),
              tfma.MetricConfig(class_name='ExampleCount'),
              tfma.MetricConfig(class_name='AUC')
              ],
              thresholds={
                  'AUC':
                      tfma.config.MetricThreshold(
                          value_threshold=tfma.GenericValueThreshold(
                              lower_bound={'value': 0.5}))}
                          )])

In [ ]:
eval_shared_models = [
  tfma.default_eval_shared_model(
      model_name='candidate', # must have this exact name
      eval_saved_model_path='serving_model_dir_150_steps/', tags=[tf.saved_model.SERVING]),
  tfma.default_eval_shared_model(
      model_name='baseline', # must have this exact name
      eval_saved_model_path='serving_model_dir_2000_steps/', tags=[tf.saved_model.SERVING]),
]

eval_result = tfma.run_model_analysis(
    eval_shared_models,
    eval_config=eval_config_threshold,
    data_location=_EVAL_DATA_FILE,
    output_path="./eval_threshold",slice_spec = slices)



In [ ]:
tfma.load_validation_result('./eval_threshold')

In [ ]:
tfma.view.render_slicing_metrics(eval_result)

## Fairness indicators

In [ ]:
# https://github.com/tensorflow/tensorboard/blob/master/docs/fairness-indicators.md
# needs environment without WIT,but with TF2.1, TFX
!pip install tensorboard_plugin_fairness_indicators

In [6]:
eval_config_fairness=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
        metrics_specs=[
              tfma.MetricsSpec(metrics=[
                  tfma.MetricConfig(class_name='BinaryAccuracy'),
                  tfma.MetricConfig(class_name='ExampleCount'),
                  tfma.MetricConfig(class_name='FalsePositives'),
                  tfma.MetricConfig(class_name='TruePositives'),
                  tfma.MetricConfig(class_name='FalseNegatives'),
                  tfma.MetricConfig(class_name='TrueNegatives'),
                  tfma.MetricConfig(class_name='FairnessIndicators', config='{"thresholds":[0.25, 0.5, 0.75]}')
              ])])

In [7]:
eval_result = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model,
    eval_config=eval_config_fairness,
    data_location=_EVAL_DATA_FILE,
    output_path="./eval_result_fairness",
    file_format='tfrecords',
    slice_spec = slices)

In [8]:
from tensorboard_plugin_fairness_indicators import summary_v2

In [9]:
writer = tf.summary.create_file_writer('./fairness_indicator_logs')
with writer.as_default():
    summary_v2.FairnessIndicators('./eval_result_fairness', step=1)
writer.close()

In [10]:
%load_ext tensorboard
%tensorboard --logdir=./fairness_indicator_logs

## The What-If Tool

In [5]:
from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

In [6]:
eval_data = tf.data.TFRecordDataset(_EVAL_DATA_FILE)

In [7]:
eval_examples = [tf.train.Example.FromString(d.numpy()) for d in eval_data.take(1000)]

In [8]:
model = tf.saved_model.load(export_dir=_MODEL_DIR)

In [9]:
def predict(examples):
    preds = model.signatures['serving_default'](examples=tf.constant([example.SerializeToString() for example in examples]))
    return preds['outputs'].numpy()

In [10]:
config_builder = WitConfigBuilder(eval_examples).set_custom_predict_fn(predict)

In [12]:
WitWidget(config_builder)

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'are_sequence_examples': False, 'inferenc…

### Debugging

In [ ]:
# works with 2.1
!pip show tensorflow

In [ ]:
# works with >0.21.3
!pip show tensorflow_model_analysis

In [ ]:
# works with 1.6.0
!pip show witwidget

In [4]:
# may need to run this every time
!jupyter nbextension install --py --symlink --sys-prefix witwidget

!jupyter nbextension enable witwidget --py --sys-prefix 

# then refresh browser page

Installing /Users/i854694/.virtualenvs/pipelines/lib/python3.7/site-packages/witwidget/static -> wit-widget
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable witwidget --py --sys-prefix
    
Enabling notebook extension wit-widget/extension...
      - Validating: OK


In [ ]:
# may need to run this every time

!jupyter nbextension enable --py widgetsnbextension --sys-prefix
  
!jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
  
!jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix

# then refresh browser page

In [ ]:
!pip install widgetsnbextension

In [ ]:
!pip install -U ipywidgets

In [ ]:
!pip install jupyter_nbextensions_configurator

In [ ]:
!jupyter nbextension list

In [ ]:
!jupyter serverextension list